In [1]:
from bs4 import BeautifulSoup
import urllib.request
from urllib.request import urlopen
from urllib.parse import quote_plus
import requests

import warnings
warnings.filterwarnings('ignore')

import os
import selenium
from selenium import webdriver
import re
from datetime import datetime as dt, timedelta
import datetime
from tqdm import tqdm

import pandas as pd
import time
import math
import numpy as np

from dateutil.parser import parse
from selenium.webdriver import Chrome, ChromeOptions
import lxml
import lxml.html

In [2]:
comp_list = pd.read_excel('./data/comp_list.xlsx').iloc[:,1:]
comp_list

,서비스업,제조/화학,의료/제약/복지,유통/무역/운송,교육업,건설업,IT/웹/통신,미디어/디자인,은행/금융업,기관/협회
0,스타벅스커피코리아,삼성전자,한미약품,씨제이올리브영,해커스 교육그룹,한국토지주택공사,쿠팡,씨제이씨지브이,국민건강보험공단,대한무역투자진흥공사
1,한국맥도날드,엘지전자,대웅제약,에프알엘코리아,동화세상에듀코,한전KPS,㈜티몬,롯데시네마,새마을금고,삼일회계법인
2,롯데리아,엘지디스플레이,종근당,한샘,에스티유니타스,현대엔지니어링,위메프,씨제이이앤엠,근로복지공단,한영회계법인
3,하나투어,현대자동차,녹십자,이마트,에듀윌,현대건설,삼성에스디에스,교원,농협은행,안진회계법인
4,유베이스,엘지이노텍,한국유나이티드제약,홈플러스,대교,대우건설,엘지씨엔에스,문화방송,중소기업은행,삼정회계법인
...,...,...,...,...,...,...,...,...,...,...
95,마이크임팩트,메디앙스,펜믹스,올림푸스한국,창의와탐구,평화엔지니어링,데이터스트림즈,아이코닉스,예금보험공사,서울산업진흥원
96,부킹닷컴코리아,한국가스공사,종근당바이오,필립스코리아,하이컨시,비엔에이치,위메이드,엠디마케팅,카카오페이,한국식품연구원
97,한국기업데이터,타이코에이엠피,경희의료원,트라이씨클,글로벌콘텐츠리퍼블릭,세아네트웍스,이카운트,한국학술정보,제이비우리캐피탈,법무법인세종
98,바이오톡스텍,에스케이케미칼,세이브더칠드런코리아,브리티쉬아메리칸토바코코리아,제이엠커리어,경남기업,에스케이브로드밴드,한국우편사업진흥원,엔에이치농협캐피탈,세종문화회관


In [6]:
comp_score = pd.DataFrame(columns=['category','name','average','career','wlb','money','culture'])

In [4]:
comp_list_pre = pd.DataFrame(columns=['category','name'])
for i in comp_list.columns:
    t = pd.DataFrame({'category':i, 'name':comp_list[i]})
    comp_list_pre = pd.concat([comp_list_pre, t], axis=0)
    
comp_list_pre = comp_list_pre.reset_index(drop=True)

In [5]:
comp_list_pre

,category,name
0,서비스업,스타벅스커피코리아
1,서비스업,한국맥도날드
2,서비스업,롯데리아
3,서비스업,하나투어
4,서비스업,유베이스
...,...,...
995,기관/협회,서울산업진흥원
996,기관/협회,한국식품연구원
997,기관/협회,법무법인세종
998,기관/협회,세종문화회관


In [ ]:
op = ChromeOptions()
# op.add_argument('--headless')
op.add_argument('--no-sandbox')
op.add_argument('--disable-dev-shm-usage')

browser = Chrome('./chromedriver.exe', options=op)
# browser.get('https://www.teamblind.com/kr/')

# login = browser.find_element_by_xpath('//*[@id="wrap"]/header/div/div/div/a[2]')
# login.click()

# time.sleep(15)

not_list = []
for i in tqdm(range(len(comp_list_pre))):
    try:
        cate = comp_list_pre.loc[i, 'category']
        name = comp_list_pre.loc[i, 'name']
        url = 'https://www.teamblind.com/kr/company/' + name + '/reviews?page=1'
        browser.get(url)
        time.sleep(0.5)
        html = browser.page_source
        root = lxml.html.fromstring(html)

        average = root.cssselect('#wrap > section > div > div > div.cpctw > div > div > div > div > section:nth-child(1) > div > div.rating_stars > div.rating_no > strong')[0].text_content().replace('Rating Score','')
        career = root.cssselect('#wrap > section > div > div > div.cpctw > div > div > div > div > section:nth-child(1) > div > div.rating_cate > ul > li:nth-child(1) > strong')[0].text_content()
        wlb = root.cssselect('#wrap > section > div > div > div.cpctw > div > div > div > div > section:nth-child(1) > div > div.rating_cate > ul > li:nth-child(2) > strong')[0].text_content()
        money = root.cssselect('#wrap > section > div > div > div.cpctw > div > div > div > div > section:nth-child(1) > div > div.rating_cate > ul > li:nth-child(3) > strong')[0].text_content()
        culture = root.cssselect('#wrap > section > div > div > div.cpctw > div > div > div > div > section:nth-child(1) > div > div.rating_cate > ul > li:nth-child(4) > strong')[0].text_content()
        board = root.cssselect('#wrap > section > div > div > div.cpctw > div > div > div > div > section:nth-child(1) > div > div.rating_cate > ul > li:nth-child(5) > strong')[0].text_content()
        data_to_insert = {
            'category': cate, 'name': name, 'average': average, 'career': career, 'wlb': wlb, 'money': money,
            'culture': culture, 'board': board
        }
    
        comp_score = comp_score.append(data_to_insert, ignore_index=True)
        
    except:
        not_list.append(name)
        print(name)

        pass

    time.sleep(0.5)

  0%|▏                                                                              | 2/1000 [00:11<1:31:19,  5.49s/it]

롯데리아 


  1%|▊                                                                             | 11/1000 [00:47<1:08:38,  4.16s/it]

씨제이푸드빌


  1%|█                                                                               | 13/1000 [00:52<56:08,  3.41s/it]

...

 28%|██████████████████████                                                         | 279/1000 [16:21<22:33,  1.88s/it]

동성제약


 28%|██████████████████████                                                         | 280/1000 [16:22<19:48,  1.65s/it]

영진약품


 28%|██████████████████████▏                                                        | 281/1000 [16:23<18:07,  1.51s/it]

In [13]:
comp_score

,category,name,homepage,upjong,geo,foundation,employee,sal_min,sal_mid,sal_max,bonus_min,bonus_mid,bonus_max
0,서비스업,스타벅스커피코리아,www.istarbucks.co.kr,외식/프랜차이즈,서울특별시 중구,1997,"10,000+(명)","2,100만원","3,326만","8,100만원",30만원,400만,"2,000만원"
1,서비스업,한국맥도날드,www.mcdonalds.co.kr,외식/프랜차이즈,서울특별시 종로구,1986,"1,001~5,000(명)","2,800만원","3,656만",1.3억원,20만원,300만,"1,500만원"
2,서비스업,하나투어,www.hanatourcompany.com,여행사/여행서비스업,서울특별시 종로구,1993,"1,001~5,000(명)","2,304만원","4,495만",1.0억원,30만원,200만,"3,000만원"
3,서비스업,유베이스,www.ubase.co.kr,사업지원/시설관리/임대,서울특별시 용산구,1999,"10,000+(명)","2,191만원","3,789만","7,374만원",80만원,300만,540만원
4,서비스업,트랜스코스모스코리아,www.trans-cosmos.co.kr,고용알선/인력공급업,서울특별시 용산구,2001,"5,001~10,000(명)","2,219만원","3,870만","7,312만원",50만원,120만,200만원
5,서비스업,세스코,www.cesco.co.kr,사업지원/시설관리/임대,서울특별시 강동구,1977,"1,001~5,000(명)","2,400만원","3,550만","8,050만원",50만원,450만,"1,450만원"
6,서비스업,아워홈,www.ourhome.co.kr,식음료 제조업,서울특별시 강남구,2000,"5,001~10,000(명)","2,286만원","4,300만","8,500만원",50만원,300만,"1,800만원"
7,서비스업,에스텍시스템,s-tec.co.kr,경비/경호/탐정업,서울특별시 강남구,1999,"10,000+(명)","2,184만원","4,120만","7,624만원",100만원,940만,"3,000만원"
8,서비스업,스타벅스커피코리아,www.istarbucks.co.kr,외식/프랜차이즈,서울특별시 중구,1997,"10,000+(명)","2,100만원","3,326만","8,100만원",30만원,400만,"2,000만원"
9,서비스업,한국맥도날드,www.mcdonalds.co.kr,외식/프랜차이즈,서울특별시 종로구,1986,"1,001~5,000(명)","2,800만원","3,656만",1.3억원,20만원,300만,"1,500만원"
